In [1]:
# Creates new models into the DB from an excel sheet
# Yes that's overkill for single records
import sqlite3
import pandas as pd
import hashing

conn = sqlite3.connect('app_db.sqlite')
cur = conn.cursor()

TargetExcelFilename = 'models.xlsx'


In [4]:
def GetNewKeyPhrase():
  sql_query = "SELECT NewKeyPhrase FROM NewKeyPhrase;";
  cur.execute(sql_query)
  row = cur.fetchone()
  NewKeyPhrase = row[0]
  #print(NewKeyPhrase)
  if NewKeyPhrase == '' : # if the view returns a blank row because it randomly selected a used ID
    NewKeyPhrase = GetNewKeyPhrase()
  sql_query = "INSERT INTO AppUsedId (AppUsedId) VALUES (?);";
  cur.execute(sql_query, [NewKeyPhrase])
  conn.commit()
  return NewKeyPhrase

toe.tomorrow.flower


In [5]:
df = pd.read_excel(TargetExcelFilename)
df['ModelKeyPhrase'] = df.apply(lambda row: GetNewKeyPhrase() if pd.isnull(row['ModelKeyPhrase']) else row['ModelKeyPhrase'], axis=1)
df.fillna('', inplace=True)

#df

,ModelKeyPhrase,ModelName,ModelConceptLevel,ModelType
0,jam,Enterprise KYC Model,Logical,
1,laugh.idea.vegetable,Zenith KYC Form,Logical,Informational


In [9]:
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'ModelHashKey'
  , columns = ['ModelKeyPhrase']
)
df = hashing.add_md5_hash_column(
  df
  , md5_column_name = 'HashDiff'
  , columns = ['ModelName','ModelConceptLevel','ModelType']
)
df['RecordSource'] = df.apply(lambda row: 'Py.CreateModel', axis=1)

#df

,ModelKeyPhrase,ModelName,ModelConceptLevel,ModelType,ModelHashKey,HashDiff,RecordSource
0,jam,Enterprise KYC Model,Logical,,5275cb415e5bc3948e8f2cd492859f26,a03de494cb50136fdf2ba4489e8bf7c1,Py.CreateModel
1,laugh.idea.vegetable,Zenith KYC Form,Logical,Informational,cb5696195f0937d19ef22ef01220e092,a3692cc9ae7283777c9d356ab9e4b4ea,Py.CreateModel


In [17]:
# Clear down and Write to staging
conn.execute("DELETE FROM stg_Py_CreateModel")
conn.commit()
df.to_sql('stg_Py_CreateModel', conn, if_exists='append', index=False)


2

In [18]:
# Write to the Hub
sql_query = """
INSERT INTO rv_h_Model
(
    ModelHashKey
  , LoadDate
  , RecordSource
  , ModelKeyPhrase
)
SELECT DISTINCT
    ModelHashKey
  , LoadDate
  , RecordSource
  , ModelKeyPhrase
FROM
  stg_Py_CreateModel
WHERE
  ModelHashKey NOT IN (SELECT ModelHashKey FROM rv_h_Model)
""";
conn.execute(sql_query)
conn.commit()

In [19]:
# Write to the Satellite
sql_query = """
INSERT INTO rv_s_Model
(
    ModelHashKey
  , LoadDate
  , RecordSource
  , HashDiff
  , ModelName
  , ModelConceptLevel
  , ModelType
)
SELECT DISTINCT
    stg.ModelHashKey
  , stg.LoadDate
  , stg.RecordSource
  , stg.HashDiff
  , stg.ModelName
  , stg.ModelConceptLevel
  , stg.ModelType
FROM
  stg_Py_CreateModel AS stg
  LEFT OUTER JOIN rv_s_Model AS sat ON (
    stg.ModelHashKey = sat.ModelHashKey
    AND sat.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_Model AS z
      WHERE z.ModelHashKey = sat.ModelHashKey
    )
  )
WHERE
  (
    sat.HashDiff != stg.HashDiff
    OR sat.HashDiff IS NULL
  )
""";
conn.execute(sql_query)
conn.commit()